In [1]:
import pymongo as pm
from pymongo import MongoClient
from bson import Code
# client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
stockDB = client.stock
companyDB = client.companyData
stockCol = stockDB.stocks
companyCol = companyDB.companies

import warnings
warnings.filterwarnings('ignore')

In [4]:
#checking the existing index info on the company collection

companyCol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'companyData.companies'}}

In [9]:
%%time

list(stockCol.aggregate([
    {"$sort": {"stock_symbol": 1}},
    {"$limit": 1}
]))

Wall time: 14.8 s


[{'_id': ObjectId('4d094f58c96767d7a0099d49'),
  'exchange': 'NASDAQ',
  'stock_symbol': 'AACC',
  'date': '2008-03-07',
  'open': 8.4,
  'high': 8.75,
  'low': 8.08,
  'close': 8.55,
  'volume': 275800,
  'adj close': 8.55}]

In [12]:
"""
db.stocks.createIndex(
  { stock_symbol: 1 } ,
  { name: "index on stock symbol" }
)
"""
stockCol.create_index([("stock_symbol", pm.ASCENDING)], name= "index on stock symbol")

'index on stock symbol'

In [13]:
stockCol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'stock.stocks'},
 'index on stock symbol': {'v': 2,
  'key': [('stock_symbol', 1)],
  'ns': 'stock.stocks'}}

In [15]:
%%time

list(stockCol.aggregate([
    {"$sort": {"stock_symbol": 1}},
    {"$limit": 1}
]))

Wall time: 74 ms


[{'_id': ObjectId('4d094fbbc96767d7a0326250'),
  'exchange': 'NASDAQ',
  'stock_symbol': 'YTEC',
  'date': '2005-02-18',
  'open': 5.6,
  'high': 5.63,
  'low': 5.5,
  'close': 5.5,
  'volume': 105000,
  'adj close': 5.5}]

As we can see our query finished in less than one second which previously took 15 secs.

In [27]:
%%time

#even reversing the order will be fast

list(stockCol.aggregate([
    {"$sort": {"stock_symbol": -1}},
    {"$limit": 1}
]))

Wall time: 2.04 ms


[{'_id': ObjectId('4d094fbbc96767d7a0326250'),
  'exchange': 'NASDAQ',
  'stock_symbol': 'YTEC',
  'date': '2005-02-18',
  'open': 5.6,
  'high': 5.63,
  'low': 5.5,
  'close': 5.5,
  'volume': 105000,
  'adj close': 5.5}]

In [36]:
%%time

list(companyCol.find({"name" : "Microsoft"}))

Wall time: 188 ms


[{'_id': ObjectId('52cdef7c4bab8bd67529809d'),
  'name': 'Microsoft',
  'permalink': 'microsoft',
  'crunchbase_url': 'http://www.crunchbase.com/company/microsoft',
  'homepage_url': 'http://www.microsoft.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Microsoft',
  'category_code': 'software',
  'number_of_employees': 90000,
  'founded_year': 1974,
  'founded_month': 4,
  'founded_day': 4,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': '',
  'tag_list': 'bill-gates, xbox, windows, vista, xp',
  'alias_list': 'msn',
  'email_address': '',
  'phone_number': '',
  'description': '',
  'created_at': 'Sat Nov 03 12:45:12 UTC 2007',
  'updated_at': 'Fri Dec 27 17:02:28 UTC 2013',
  'overview': '<p>Microsoft, founded in 1975 by Bill Gates and Paul Allen, is a veteran software company, best known for its <a href="/product/windows" title="Microsoft Windows" rel="nofollow">Microsoft Windows</a> operating system and the 

In [42]:
companyCol.create_index([("name", pm.ASCENDING), ("permalink", pm.ASCENDING)], name= "company index on name and link")

'company index on name and link'

In [45]:
%%time

list(companyCol.find({"name" : "AdventNet"}))

Wall time: 2 ms


[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'permalink': 'abc3',
  'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
  'homepage_url': 'http://adventnet.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'manageengine',
  'category_code': 'enterprise',
  'number_of_employees': 600,
  'founded_year': 1996,
  'deadpooled_year': 2,
  'tag_list': '',
  'alias_list': 'Zoho ManageEngine ',
  'email_address': 'pr@adventnet.com',
  'phone_number': '925-924-9500',
  'description': 'Server Management Software',
  'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
  'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
  'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>

In [48]:
%%time

len(list(companyCol.find({"relationships.title": {"$in" : ["Director of Engineering", "CEO and Co-Founder"]}}, {"name": 1})))

Wall time: 59 ms


196

In [49]:
companyCol.create_index([("relationships.title", pm.ASCENDING)], name= "company index on emp array")

'company index on emp array'

In [57]:
%%time

len(list(companyCol.find({"relationships.title": {"$in" : ["Director of Engineering", "CEO and Co-Founder"]}}, {"name": 1})))

Wall time: 6.04 ms


196

In [60]:
companyCol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'companyData.companies'},
 'company index on name and link': {'v': 2,
  'key': [('name', 1), ('permalink', 1)],
  'ns': 'companyData.companies'},
 'company index on emp array': {'v': 2,
  'key': [('relationships.title', 1)],
  'ns': 'companyData.companies'}}

In [62]:
companyCol.drop_index('company index on emp array')
companyCol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'companyData.companies'},
 'company index on name and link': {'v': 2,
  'key': [('name', 1), ('permalink', 1)],
  'ns': 'companyData.companies'}}

# Partial Index

Partial index gives you a way to select certain doc instead of all to index upon. we can specify filters while creating the index.

One of the use cases to think about this is, let's say you've a SAAS product and you're giving them free trials. Once the free trial ends,
user shouldn't be able to access platform . So naturally most of the queries would be involving checking whether the user has access.

So we can index with filter like 'useraccess' : 1

# Sparse Index

Sparse indexes only contain entries for documents that have the indexed field, even if the index field contains a null value. The index skips over any document that is missing the indexed field. The index is “sparse” because it does not include all documents of a collection. By contrast, non-sparse indexes contain all documents in a collection, storing null values for those documents that do not contain the indexed field.

for ex:

db.addresses.createIndex( { "xmpp_id": 1 }, { sparse: true } )

However, Partial Index is considered a super set of Sparse and the same can be acheived by partial index with filter.

# TTL Index

When we want to delete docs from a collection after certain time, we can make use of TTL index. The index is obviously made on 
date field and has param to expire the doc after certain seconds


```python
db.adminCommand({setParameter:1, ttlMonitorSleepSecs: 5}); // run TTL monitor in every 5 seconds
sleep(60000); // wait a minute for new sleep interval to take effect

db.ttldemo.drop();
db.ttldemo.createIndex({created_at: 1}, {expireAfterSeconds: 0});

// Create document with Random expiry time ranging from 0 to 60 seconds in future
for(var i=0; i< 10000; i++){
    db.ttldemo.insert({
            created_at: new Date(Date.now() + Math.floor(Math.random()*60000))
            })
    }
for(var i=0; i<13;i++){
    print(db.ttldemo.count());
    sleep(5*1000); //wait 5 seconds
    }
db.adminCommand({setParameter:1, ttlMonitorSleepSecs: 60});
```

Output for the above is


9833

9077

8299

7459

6653

5785

4936

4123

3263

2400

1536

659

42


As we can see docs are getting deleted once it expies

In [ ]:
References:

http://hassansin.github.io/working-with-mongodb-ttl-index
https://docs.mongodb.com/manual/core/index-intersection/

